# VeronaCard Next‑POI Prediction – Metrics & Exploratory Analysis

*Generated automatically on 2025-06-25 13:01 UTC*

# 🎯 Obiettivi dell'Analisi

Questo notebook fornisce una **valutazione completa** delle performance del modello di predizione next-POI implementato in `veronacard_mob.py`, attraverso un framework di analisi strutturato e riproducibile.

## 📋 Pipeline di Analisi

### 🔄 **1. Data Loading & Preprocessing**
- **Caricamento automatico** di tutti i file CSV di predizione (`*_pred_*.csv`)
- **Parsing robusto** con gestione errori per file malformati
- **Estrazione automatica** dell'anno dai nomi file (es. `dati_2016_pred_model.csv` → 2016)
- **Validazione** delle predizioni (filtro per liste con esattamente 5 elementi)

### 📊 **2. Evaluation Metrics**
Calcolo di **quattro metriche standard** per sistemi di raccomandazione:
- **🎯 Top-1 Accuracy**: Precisione della predizione principale
- **📈 Top-5 Hit Rate**: Presenza del POI corretto nella top-5
- **⚡ Mean Reciprocal Rank (MRR)**: Qualità del ranking delle predizioni
- **🗂️ Catalogue Coverage**: Diversità delle raccomandazioni

### 📈 **3. Multi-level Visualization**
- **📊 Global metrics**: Performance aggregate su tutto il dataset
- **📅 Temporal analysis**: Trend evolutivo per anno (2016→2020)
- **🎨 Interactive charts**: Grafici comparativi e heat-maps

### 🔍 **4. Advanced Error Analysis Framework**
Sistema modulare ed estendibile per analisi approfondita:
- **🚨 Worst-case analysis**: Identificazione delle coppie POI più problematiche
- **🔥 Confusion matrices**: Visualizzazione degli errori sui POI più frequenti
- **🧠 Explainability**: Analisi dei pattern testuali che causano errori (LIME/SHAP-ready)
- **📊 Temporal patterns**: Analisi stagionale e trend temporali degli errori

## 🏗️ Caratteristiche Architetturali

### **✅ Robustezza**
- **Error handling** completo per file CSV corrotti
- **Graceful degradation** per dipendenze mancanti (LIME, etc.)
- **Validation** automatica dei dati di input

### **🔧 Flessibilità**
- **Path-agnostic**: Funziona con CSV nella cartella corrente o tramite `DATA_DIR`
- **Modular design**: Sezioni indipendenti eseguibili singolarmente
- **Extensible framework**: Facile aggiungere nuove metriche o analisi

### **📊 Riproducibilità**
- **Timestamp automatico** nel titolo
- **Seed fisso** per operazioni random
- **Versioning implicito** tramite nomi file con data

---

## 🚀 Quick Start

```python
# Esecuzione standard - CSV nella cartella 'results/'
# Nessuna configurazione richiesta

# Esecuzione personalizzata
DATA_DIR = "/path/to/your/csv/files"  # opzionale
```

## 📁 Struttura File Attesa

```
project_root/
├── notebook.ipynb           # questo notebook
├── results/                 # directory predefinita
│   ├── dati_2016_pred_model1.csv
│   ├── dati_2017_pred_model1.csv
│   └── ...
└── veronacard_mob.py       # script di generazione predizioni
```

---

> **💡 Pro Tip**: Il notebook è progettato per **fail-safe operation** — anche se alcuni file sono corrotti o alcune librerie mancano, l'analisi procede con le parti disponibili e fornisce sempre risultati utili.

In [ ]:
from pathlib import Path
import ast, json, re, csv
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.dpi'] = 120
plt.rcParams['axes.grid'] = True

# Configuration - easily switch between different result directories
#DATA_DIR = "../results_mixtral_8x7b_with_geom/" 
#DATA_DIR = "../results/results_mixtral_8x7b_with_geom/" 
DATA_DIR = "../results/qwen2.5_14b//with_geom/" 
#DATA_DIR = "../results/results_mixtral_8x7b_base_version"
#DATA_DIR = "../results_parziali_sequenziali_ministral/" 
#DATA_DIR = "../result_run_completa_with_geom_srv_univr/" #llama 3.1:8b
#DATA_DIR = "../results"

print(f"📁 Loading data from: {DATA_DIR}")
print(f"🛡️ Robust CSV parsing enabled")

## 1. 🛡️ Robust Data Loading

Caricamento robusto dei dati con gestione automatica di:
- **Escaped quotes**: `""Basilica di Sant'Anastasia""` → `'Basilica di Sant\'Anastasia'`
- **CSV malformati**: Parsing con multiple strategie di fallback
- **Predizioni incomplete**: Filtro automatico per dati validi
- **Error tracking**: Monitoraggio della qualità dei dati

In [ ]:
def robust_parse_prediction(x, debug=False):
    """
    Ultra-robust prediction parsing with multiple fallback strategies
    
    Handles the specific issue in CSV files where POI names contain escaped quotes
    like: ""Basilica di Sant'Anastasia""
    """
    if not isinstance(x, str) or not x.strip():
        return []
    
    x = x.strip()
    
    # Strategy 1: Try ast.literal_eval first (fastest for well-formed data)
    try:
        result = ast.literal_eval(x)
        if isinstance(result, list):
            return [str(item) for item in result]
        else:
            return [str(result)]
    except (ValueError, SyntaxError):
        pass
    
    # Strategy 2: Handle escaped quotes (main issue in CSV files)
    try:
        # Replace double escaped quotes with single quotes
        # ""Basilica di Sant'Anastasia"" -> 'Basilica di Sant\'Anastasia'
        fixed = re.sub(r'""([^"]*?)""', r"'\1'", x)
        
        result = ast.literal_eval(fixed)
        if isinstance(result, list):
            return [str(item) for item in result]
        else:
            return [str(result)]
    except (ValueError, SyntaxError):
        pass
    
    # Strategy 3: Manual parsing with regex
    try:
        if x.startswith('[') and x.endswith(']'):
            # Extract content between various quote types
            patterns = [
                r"'([^']*?)'(?:,|\\])",  # Single quotes
                r'"([^"]*?)"(?:,|\\])',   # Double quotes  
                r'""([^"]*?)""(?:,|\\])'  # Escaped double quotes
            ]
            
            result = []
            for pattern in patterns:
                matches = re.findall(pattern, x + ',')
                result.extend(matches)
            
            # Remove duplicates while preserving order
            seen = set()
            unique_result = []
            for item in result:
                if item not in seen:
                    seen.add(item)
                    unique_result.append(item)
            
            if unique_result:
                return unique_result
    except Exception:
        pass
    
    # Strategy 4: JSON-like parsing
    try:
        json_like = x.replace("'", '"')  # Convert single quotes to double
        result = json.loads(json_like)
        if isinstance(result, list):
            return [str(item) for item in result]
        else:
            return [str(result)]
    except (json.JSONDecodeError, ValueError):
        pass
    
    # Strategy 5: Simple comma splitting (last resort)
    try:
        if ',' in x:
            clean = x.strip('[]').split(',')
            result = []
            for item in clean:
                item = item.strip().strip('"\'')
                if item:
                    result.append(item)
            if result:
                return result
    except Exception:
        pass
    
    # Final fallback: empty list with optional warning
    if debug:
        print(f"⚠️  Could not parse: '{x[:50]}{'...' if len(x) > 50 else ''}'")
    
    return []


def robust_csv_reader(file_path, debug=False):
    """
    Robust CSV reader with multiple parsing strategies.
    Handles various CSV formats including those with additional columns
    like processing_time, status, etc.
    """
    try:
        # Try standard pandas CSV reader first
        df = pd.read_csv(file_path)
        
        # Check if we have the minimum required columns
        required_columns = ['card_id', 'prediction', 'ground_truth']
        if all(col in df.columns for col in required_columns):
            if debug:
                print(f"✅ Standard CSV read: {len(df)} rows, columns: {list(df.columns)}")
            return df
        else:
            missing_cols = [col for col in required_columns if col not in df.columns]
            if debug:
                print(f"⚠️  Missing required columns: {missing_cols}")
            
    except pd.errors.ParserError as e:
        if debug:
            print(f"⚠️  Standard read failed: {e}")
    
    # Fallback: try with different parsing options
    try:
        df = pd.read_csv(file_path, on_bad_lines='skip', engine='python')
        
        # Check for required columns again
        required_columns = ['card_id', 'prediction', 'ground_truth']
        if all(col in df.columns for col in required_columns):
            if debug:
                print(f"✅ Fallback CSV read: {len(df)} rows, columns: {list(df.columns)}")
            return df
        else:
            if debug:
                print(f"⚠️  Fallback read missing columns")
                
    except Exception as e2:
        if debug:
            print(f"❌ All CSV reads failed: {e2}")
    
    # Final fallback: return empty DataFrame
    return pd.DataFrame()


# Find and load all CSV files
csv_files = [Path(p) for p in glob.glob(os.path.join(DATA_DIR, '*_pred_*.csv'))]
csv_files = sorted(csv_files)

if not csv_files:
    raise FileNotFoundError(f"No CSV files matching *_pred_*.csv found in {DATA_DIR}")

print(f"📂 Found {len(csv_files)} CSV files to process")

# Process files with robust parsing and error tracking
dfs = []
parse_errors = 0
total_rows_processed = 0
skipped_rows = 0
files_with_issues = []

for i, fp in enumerate(csv_files, 1):
    print(f"\\n📄 Processing file {i}/{len(csv_files)}: {fp.name}")
    
    # Load CSV with robust reader
    df = robust_csv_reader(fp, debug=True)
    
    if df.empty:
        print(f"   ⚠️  Skipped empty/corrupted file")
        files_with_issues.append(fp.name)
        continue
    
    # Handle missing columns gracefully
    if 'ground_truth' not in df.columns:
        print(f"   ⚠️  No ground_truth column found, skipping file")
        files_with_issues.append(fp.name)
        continue
        
    if 'prediction' not in df.columns:
        print(f"   ⚠️  No prediction column found, skipping file")
        files_with_issues.append(fp.name)
        continue
    
    # Handle optional columns with defaults
    if 'hit' not in df.columns:
        df['hit'] = False  # Default to False if not present
        print(f"   📝 Added default 'hit' column")
    
    if 'cluster' not in df.columns:
        df['cluster'] = 0  # Default cluster
        print(f"   📝 Added default 'cluster' column")
    
    if 'history' not in df.columns:
        df['history'] = "[]"  # Empty history
        print(f"   📝 Added default 'history' column")
    
    if 'current_poi' not in df.columns:
        df['current_poi'] = ""  # Empty current POI
        print(f"   📝 Added default 'current_poi' column")
    
    # Extract year from filename (e.g., dati_2016_pred_...)
    year_token = next((part for part in fp.stem.split('_')
                       if part.isdigit() and len(part) == 4), None)
    df['year'] = int(year_token) if year_token else np.nan
    
    # Filter out rows with empty ground_truth
    original_count = len(df)
    df = df[df['ground_truth'].notna() & (df['ground_truth'].astype(str).str.strip() != "")].copy()
    empty_gt_rows = original_count - len(df)
    
    if empty_gt_rows > 0:
        print(f"   📉 Filtered out {empty_gt_rows} rows with empty ground_truth")
        skipped_rows += empty_gt_rows
    
    if df.empty:
        print(f"   ⚠️  No valid rows remaining after filtering")
        files_with_issues.append(fp.name)
        continue
    
    # Robust prediction parsing
    print(f"   🔧 Parsing {len(df)} prediction entries...")
    
    df['prediction_list'] = df['prediction'].apply(robust_parse_prediction)
    
    # Count parsing issues
    empty_predictions = (df['prediction_list'].apply(len) == 0).sum()
    if empty_predictions > 0:
        parse_errors += empty_predictions
        print(f"   ⚠️  {empty_predictions} entries could not be parsed")
    
    # Filter for valid predictions (exactly 5 elements)
    df_valid = df[df['prediction_list'].apply(len) == 5].copy()
    rows_kept = len(df_valid)
    rows_dropped = len(df) - rows_kept
    
    if rows_dropped > 0:
        skipped_rows += rows_dropped
        print(f"   📉 Filtered out {rows_dropped} rows (wrong prediction count)")
    
    print(f"   ✅ Kept {rows_kept} valid rows")
    
    if not df_valid.empty:
        dfs.append(df_valid)
        total_rows_processed += rows_kept

# Combine all DataFrames
if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
    print(f"\\n🎉 Successfully processed {len(csv_files)} files:")
    print(f"   📊 Total valid rows: {total_rows_processed:,}")
    print(f"   ⚠️  Parse errors: {parse_errors:,}")
    print(f"   📉 Skipped rows: {skipped_rows:,}")
    
    if files_with_issues:
        print(f"   ⚠️  Files with issues: {len(files_with_issues)}")
        for fname in files_with_issues[:5]:  # Show first 5
            print(f"      • {fname}")
        if len(files_with_issues) > 5:
            print(f"      • ... and {len(files_with_issues) - 5} more")
    
    success_rate = (total_rows_processed/(total_rows_processed+skipped_rows+parse_errors))*100
    print(f"   ✅ Success rate: {success_rate:.1f}%")
    
    # Data quality summary
    print(f"\\n📈 Data Quality Summary:")
    years_available = sorted(df_all['year'].dropna().unique().astype(int).tolist())
    print(f"   • Years covered: {years_available}")
    print(f"   • Unique POI in ground truth: {df_all['ground_truth'].nunique()}")
    print(f"   • Average predictions per POI: {len(df_all) / df_all['ground_truth'].nunique():.1f}")
    
    # Check for additional columns
    extra_cols = [col for col in df_all.columns if col not in 
                  ['card_id', 'cluster', 'history', 'current_poi', 'prediction', 
                   'ground_truth', 'reason', 'hit', 'year', 'prediction_list']]
    if extra_cols:
        print(f"   • Additional columns found: {extra_cols}")
    
    # Display sample data
    print(f"\\n📋 Sample data:")
    display_cols = ['card_id', 'year', 'current_poi', 'ground_truth', 'prediction_list', 'hit']
    if 'processing_time' in df_all.columns:
        display_cols.append('processing_time')
    if 'status' in df_all.columns:
        display_cols.append('status')
    
    display(df_all[display_cols].head())
else:
    raise ValueError("No valid data could be loaded from any CSV file")

# 2. Definizione delle Metriche di Valutazione

Per valutare le performance del modello di predizione next-POI, utilizziamo quattro metriche standard nel campo dei sistemi di raccomandazione e della predizione sequenziale.

## Notazione

Sia $y_i$ il vero next-POI (ground truth) per la sequenza $i$-esima, e sia $\hat{y}_i^{(1)}, \hat{y}_i^{(2)}, \ldots, \hat{y}_i^{(k)}$ la lista **ordinata** delle $k$ raccomandazioni prodotte dal modello per quella sequenza, dove $\hat{y}_i^{(1)}$ è la predizione con confidence più alta.

---

## 📊 Top-1 Accuracy

Misura la **precisione della predizione principale** del modello.

$$\text{Acc}_{@1} = \frac{1}{N}\sum_{i=1}^{N}\mathbf{1}\!\left\{y_i = \hat{y}_i^{(1)}\right\}$$

**Interpretazione**: Percentuale di casi in cui la prima predizione del modello coincide esattamente con il POI realmente visitato.

**Range**: [0, 1], dove 1 = predizione perfetta

---

## 🎯 Top-k Hit Rate

Misura se il POI corretto è presente **tra le prime $k$ predizioni**.

$$\text{HR}_{@k} = \frac{1}{N}\sum_{i=1}^{N}\mathbf{1}\!\left\{y_i \in \{\hat{y}_i^{(1)}, \hat{y}_i^{(2)}, \ldots, \hat{y}_i^{(k)}\}\right\}$$

**Interpretazione**: Percentuale di casi in cui il POI corretto è presente nella lista delle prime $k$ raccomandazioni. Per $k=5$: "Il modello include il POI corretto nella sua top-5?"

**Range**: [0, 1], sempre $\text{HR}_{@k} \geq \text{Acc}_{@1}$

---

## ⚡ Mean Reciprocal Rank (MRR)

Considera sia la **presenza** del POI corretto nella lista che la sua **posizione**.

$$\text{MRR} = \frac{1}{N}\sum_{i=1}^{N}\frac{1}{\text{rank}_i}$$

dove $\text{rank}_i = \min\{r \mid y_i = \hat{y}_i^{(r)}\}$ è la posizione del POI corretto nella lista ordinata.

**Interpretazione**: 
- Se il POI corretto è al 1° posto → contributo = 1.0
- Se il POI corretto è al 2° posto → contributo = 0.5  
- Se il POI corretto è al 5° posto → contributo = 0.2
- Se il POI corretto non è nella top-k → contributo = 0.0

**Range**: [0, 1], dove valori più alti indicano che i POI corretti appaiono nelle prime posizioni

---

## 🗂️ Catalogue Coverage  

Misura la **diversità** delle raccomandazioni prodotte dal modello.

$$\text{Coverage} = \frac{\left|\bigcup_{i=1}^{N}\{\hat{y}_i^{(1)}, \hat{y}_i^{(2)}, \ldots, \hat{y}_i^{(k)}\}\right|}{|\mathcal{P}|}$$

dove $\mathcal{P}$ è l'insieme completo dei POI presenti nel dataset (ground truth).

**Interpretazione**: Frazione dei POI disponibili che il modello è in grado di raccomandare. 
- Coverage = 1.0: il modello raccomanda tutti i POI del catalogo
- Coverage = 0.1: il modello raccomanda solo il 10% dei POI disponibili

**Importanza**: Previene il bias verso POI molto popolari e garantisce diversità nelle raccomandazioni.

---

## 💡 Note Metodologiche

- **$N$**: Numero totale di predizioni nel dataset di test
- **$k = 5$**: Utilizziamo consistently una finestra di 5 raccomandazioni
- **Ordinamento**: Le predizioni sono ordinate per confidence/probabilità decrescente
- **Handling missing**: Se $y_i \notin \{\hat{y}_i^{(1)}, \ldots, \hat{y}_i^{(k)}\}$, contributo = 0 per tutte le metriche

Queste metriche forniscono una **valutazione completa** del modello: precision (Acc@1), recall (HR@k), ranking quality (MRR), e diversity (Coverage).

### 2.1 Helper functions & per‑row computations

In [ ]:
import json

def poi_id(x):
    """
    Converte 'x' in un identificatore hashable comparabile con ground-truth.
    • Se x è un dict          → prova a usare 'poi', 'poi_id', 'name', 'id'
    • Se x è list/tuple       → ritorna tupla ricorsiva
    • Altrimenti (str/int)    → cast a str
    """
    if isinstance(x, dict):
        for key in ('poi', 'poi_id', 'name', 'id'):
            if key in x:
                return str(x[key])
        # fallback: serializza in JSON ordinato
        return json.dumps(x, sort_keys=True)
    elif isinstance(x, (list, tuple)):
        return tuple(map(poi_id, x))
    else:
        return str(x)

# Normalizza prediction_list e ground_truth
df_all['prediction_norm']    = df_all['prediction_list'].apply(lambda lst: [poi_id(e) for e in lst])
df_all['ground_truth_norm']  = df_all['ground_truth'].apply(poi_id)

# -- metriche element-wise --------------------------------------------
df_all['hit@1'] = df_all['prediction_norm'].str[0] == df_all['ground_truth_norm']

def top_k_hit(row, k=5):
    return row['ground_truth_norm'] in row['prediction_norm'][:k]

def reciprocal_rank(row, k=5):
    try:
        rank = row['prediction_norm'][:k].index(row['ground_truth_norm']) + 1
        return 1.0 / rank
    except ValueError:
        return 0.0

df_all['hit@5'] = df_all.apply(top_k_hit, axis=1)
df_all['rr']    = df_all.apply(reciprocal_rank, axis=1)

### 2.2 Global metrics

In [ ]:
metrics_global = {
    "Top-1 Accuracy": df_all["hit@1"].mean(),
    "Top-5 Hit Rate": df_all["hit@5"].mean(),
    "MRR": df_all["rr"].mean(),
}

# Catalogue Coverage
coverage_set = {poi for preds in df_all["prediction_norm"] for poi in preds}
metrics_global["Catalogue Coverage"] = (
    len(coverage_set) / df_all["ground_truth_norm"].nunique()
)

# Visualizza in percentuale
metrics_df = (
    pd.DataFrame(metrics_global, index=["Value"])
      .T
      .style.format("{:.2%}")
)
display(metrics_df)

### 2.3 Metrics by year

In [ ]:

by_year = (
    df_all
    .groupby('year')
    .agg(
        top1=('hit@1', 'mean'),
        hit5=('hit@5', 'mean'),
        mrr=('rr', 'mean'),
        n=('card_id', 'size')
    )
    .reset_index()
    .sort_values('year')
)
display(by_year)


## 3. Visualisations

In [ ]:

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(6,4))
ax.bar(by_year['year'].astype(str), by_year['top1']*100)
ax.set_ylabel('Top‑1 Accuracy (%)')
ax.set_title('Top‑1 Accuracy (%) by Year')
plt.show()


In [ ]:

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(6,4))
ax.bar(by_year['year'].astype(str), by_year['hit5']*100)
ax.set_ylabel('Top‑5 Hit Rate (%)')
ax.set_title('Top‑5 Hit Rate (%) by Year')
plt.show()


In [ ]:

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(6,4))
ax.bar(by_year['year'].astype(str), by_year['mrr']*100)
ax.set_ylabel('MRR (%)')
ax.set_title('MRR (%) by Year')
plt.show()


### 3.1 Coverage breakdown

In [ ]:
unique_pred_poi = (
    pd.Series([poi for preds in df_all["prediction_norm"] for poi in preds])
      .nunique()
)
unique_gt_poi = df_all["ground_truth_norm"].nunique()

print(f"POI distinct in predictions: {unique_pred_poi}")
print(f"POI distinct in ground-truth: {unique_gt_poi}")
print(f"Catalogue Coverage: {unique_pred_poi/unique_gt_poi:.2%}")

## 4. Error analysis – overview

In questa sezione valutiamo **dove** e **perché** il modello sbaglia, seguendo tre filoni:

1. **Worst-performing POI pairs**  
   Scopriamo quali coppie `ground-truth → first_pred` generano il maggior numero di errori.

2. **Confusion matrix per subset**  
   Heat-map delle frequenze (o tasso d’errore) su un sotto-insieme di POI di interesse
   (ad es. i 20 più visitati o solo l’anno 2020).

3. **Explainability (SHAP / LIME)**  
   Analisi dei fattori testuali nella colonna `history` che portano a una predizione
   sbagliata. Qui mostriamo un prototipo basato su LIME; lo script è modulare
   e può essere sostituito da SHAP se usi modelli compatibili.


In [ ]:
# ---------------------------------------------------------------------
# 4.1 Worst-performing POI pairs (Top-N) – FIX
# ---------------------------------------------------------------------
# filtro righe con errore Top-1
ERR = df_all.assign(first_pred=df_all["prediction_norm"].str[0])
ERR = ERR[~ERR["hit@1"]]           # equivalente a query("`hit@1` == False")

# conteggia coppie ground-truth → first_pred
pairs = (
    ERR.groupby(["ground_truth_norm", "first_pred"])
        .size()
        .reset_index(name="errors")
        .sort_values("errors", ascending=False)
        .head(15)          # Top-15 coppie
)

display(pairs)

# barplot (opzionale)
fig, ax = plt.subplots(figsize=(8, 4))
ax.barh(
    pairs.apply(lambda r: f"{r['ground_truth_norm']} → {r['first_pred']}", axis=1),
    pairs["errors"]
)
ax.set_xlabel("Errori (count)")
ax.set_title("Worst-performing POI pairs – Top 15")
ax.invert_yaxis()
plt.show()


In [ ]:
# ---------------------------------------------------------------------
# 4.2 Confusion matrix – seleziona subset
# ---------------------------------------------------------------------
TOP_K = 20          # limitiamoci ai 20 POI più frequenti (ground-truth)
YEAR  = None        # imposta un anno (es. 2020) oppure None per all

subset = df_all.copy()
if YEAR is not None:
    subset = subset.query("year == @YEAR")

# filtra i POI più frequenti
top_poi = (
    subset["ground_truth_norm"]
           .value_counts()
           .head(TOP_K)
           .index
)
mask = subset["ground_truth_norm"].isin(top_poi) & subset["prediction_norm"].str[0].isin(top_poi)
cm_df = pd.crosstab(
    subset.loc[mask, "ground_truth_norm"],
    subset.loc[mask, "prediction_norm"].str[0],
    rownames=["True"],
    colnames=["Pred"],
    dropna=False
)

# Normalizza su righe per avere tassi d’errore
cm_norm = cm_df.div(cm_df.sum(axis=1), axis=0)

# Heat-map (matplotlib only)
fig, ax = plt.subplots(figsize=(8,6))
im = ax.imshow(cm_norm.values, aspect="auto")
ax.set_xticks(range(len(cm_norm.columns)))
ax.set_xticklabels(cm_norm.columns, rotation=90)
ax.set_yticks(range(len(cm_norm.index)))
ax.set_yticklabels(cm_norm.index)
ax.set_xlabel("Predicted POI")
ax.set_ylabel("True POI")
ax.set_title(f"Confusion matrix – top {TOP_K} POI" + (f", year {YEAR}" if YEAR else ""))
fig.colorbar(im, ax=ax, fraction=0.02, pad=0.04, label="Error rate")
plt.tight_layout()
plt.show()


In [ ]:
# ---------------------------------------------------------------------
# 4.3 Explainability 
# ---------------------------------------------------------------------
import warnings, json, numpy as np, pandas as pd
from collections import Counter
import re

print("="*60)
print("ERROR ANALYSIS - Simplified Version (No LIME required)")
print("="*60)

# ---------------------------------------------------------------------
# Analisi dei pattern testuali negli errori
# ---------------------------------------------------------------------
err_rows = df_all[~df_all["hit@1"]].copy()
correct_rows = df_all[df_all["hit@1"]].copy()

if err_rows.empty:
    print("🎉 Nessun errore Top-1 da analizzare!")
else:
    print(f"📊 Analyzing {len(err_rows):,} prediction errors vs {len(correct_rows):,} correct predictions")
    
    # Aggiungi colonne per l'analisi
    err_rows['first_pred'] = err_rows["prediction_norm"].str[0]
    correct_rows['first_pred'] = correct_rows["prediction_norm"].str[0]
    
    # ---------------------------------------------------------------------
    # 1. Analisi lunghezza history
    # ---------------------------------------------------------------------
    print("\n1️⃣ HISTORY LENGTH ANALYSIS")
    print("-" * 40)
    
    err_rows['history_length'] = err_rows['history'].astype(str).str.len()
    correct_rows['history_length'] = correct_rows['history'].astype(str).str.len()
    
    print(f"Average history length in ERRORS: {err_rows['history_length'].mean():.0f} chars")
    print(f"Average history length in CORRECT: {correct_rows['history_length'].mean():.0f} chars")
    
    # ---------------------------------------------------------------------
    # 2. Analisi parole chiave più frequenti negli errori
    # ---------------------------------------------------------------------
    print("\n2️⃣ KEYWORD ANALYSIS IN ERROR CASES")
    print("-" * 40)
    
    def extract_keywords(text, min_length=3):
        """Estrae parole significative dal testo"""
        if pd.isna(text):
            return []
        # Rimuovi caratteri speciali e converti in lowercase
        words = re.findall(r'\b[a-zA-ZÀ-ÿ]+\b', str(text).lower())
        # Filtra parole troppo corte e comuni
        stop_words = {'and', 'the', 'poi', 'via', 'del', 'dei', 'per', 'con', 'una', 'uno', 'che', 'nel', 'sul'}
        return [w for w in words if len(w) >= min_length and w not in stop_words]
    
    # Estrai keywords dalle history degli errori
    error_keywords = []
    for history in err_rows['history'].head(1000):  # Limita per performance
        error_keywords.extend(extract_keywords(history))
    
    # Estrai keywords dalle history corrette
    correct_keywords = []
    for history in correct_rows['history'].head(1000):  # Limita per performance
        correct_keywords.extend(extract_keywords(history))
    
    error_word_freq = Counter(error_keywords).most_common(15)
    correct_word_freq = Counter(correct_keywords).most_common(15)
    
    print("Most frequent words in ERROR cases:")
    for word, freq in error_word_freq:
        print(f"  {word:>15s}: {freq:>4d}")
    
    print("\nMost frequent words in CORRECT cases:")
    for word, freq in correct_word_freq:
        print(f"  {word:>15s}: {freq:>4d}")
    
    # ---------------------------------------------------------------------
    # 3. Analisi dei POI più problematici
    # ---------------------------------------------------------------------
    print("\n3️⃣ MOST PROBLEMATIC POI (Ground Truth)")
    print("-" * 40)
    
    # POI con più errori
    error_by_true_poi = (
        err_rows['ground_truth_norm']
        .value_counts()
        .head(10)
        .reset_index()
    )
    error_by_true_poi.columns = ['POI', 'Error_Count']
    
    # Calcola anche il tasso di errore
    total_by_poi = df_all['ground_truth_norm'].value_counts()
    error_by_true_poi['Total_Count'] = error_by_true_poi['POI'].map(total_by_poi)
    error_by_true_poi['Error_Rate'] = error_by_true_poi['Error_Count'] / error_by_true_poi['Total_Count']
    
    print("POI with highest error counts:")
    for _, row in error_by_true_poi.iterrows():
        print(f"  {row['POI']:>25s}: {row['Error_Count']:>3d} errors ({row['Error_Rate']:>5.1%} rate)")
    
    # ---------------------------------------------------------------------
    # 4. Analisi delle predizioni più frequenti negli errori
    # ---------------------------------------------------------------------
    print("\n4️⃣ MOST FREQUENT WRONG PREDICTIONS")
    print("-" * 40)
    
    wrong_pred_freq = err_rows['first_pred'].value_counts().head(10)
    print("Most frequently predicted (wrong) POI:")
    for poi, freq in wrong_pred_freq.items():
        print(f"  {poi:>25s}: {freq:>3d} times")
    
    # ---------------------------------------------------------------------
    # 5. Esempio dettagliato di errore
    # ---------------------------------------------------------------------
    print("\n5️⃣ DETAILED ERROR EXAMPLE")
    print("-" * 40)
    
    # Prendi il primo errore disponibile
    sample_idx = err_rows.index[0]
    sample = df_all.loc[sample_idx]
    
    print(f"Card ID: {sample.get('card_id', 'N/A')}")
    print(f"Year: {sample.get('year', 'N/A')}")
    print(f"True POI: {sample['ground_truth_norm']}")
    print(f"Predicted POI: {sample['prediction_norm'][0]}")
    print(f"Top-5 Predictions: {sample['prediction_norm']}")
    print(f"History (first 200 chars): {str(sample['history'])[:200]}...")
    
    # ---------------------------------------------------------------------
    # 6. Matrice di confusione semplificata (top POI)
    # ---------------------------------------------------------------------
    print("\n6️⃣ CONFUSION MATRIX - TOP 10 POI")
    print("-" * 40)
    
    # Prendi i 10 POI più frequenti
    top_poi = df_all['ground_truth_norm'].value_counts().head(10).index.tolist()
    
    # Filtra solo le predizioni che coinvolgono questi POI
    confusion_data = df_all[
        (df_all['ground_truth_norm'].isin(top_poi)) & 
        (df_all['prediction_norm'].str[0].isin(top_poi))
    ].copy()
    
    confusion_matrix = pd.crosstab(
        confusion_data['ground_truth_norm'],
        confusion_data['prediction_norm'].str[0],
        normalize='index'  # Normalizza per righe (% di errore per POI)
    )
    
    print("Confusion matrix (% of predictions for each true POI):")
    print("Rows = True POI, Columns = Predicted POI")
    print(confusion_matrix.round(3))
    
    # ---------------------------------------------------------------------
    # 7. Summary statistiche
    # ---------------------------------------------------------------------
    print(f"\n7️⃣ SUMMARY STATISTICS")
    print("-" * 40)
    print(f"Total predictions analyzed: {len(df_all):,}")
    print(f"Total errors (Top-1): {len(err_rows):,} ({len(err_rows)/len(df_all):.2%})")
    print(f"Unique POI in dataset: {df_all['ground_truth_norm'].nunique():,}")
    print(f"Unique POI in predictions: {len({poi for preds in df_all['prediction_norm'] for poi in preds}):,}")
    
    if len(csv_files) > 1:
        error_by_year = df_all.groupby('year')['hit@1'].agg(['count', 'mean']).reset_index()
        error_by_year['error_rate'] = 1 - error_by_year['mean']
        print(f"\nError rate by year:")
        for _, row in error_by_year.iterrows():
            if not pd.isna(row['year']):
                print(f"  {int(row['year'])}: {row['error_rate']:.2%} ({int(row['count'])} predictions)")

print("\n" + "="*60)
print("ANALYSIS COMPLETE")
print("This analysis doesn't require LIME but provides valuable insights into prediction errors.")
print("="*60)